# Importing and retrieving data

In [ ]:
import DataRetriever as dr
import pandas as pd
import numpy as np
import plotly.express as px

retriever = dr.DataRetriever()

# Load the metadata
metadata = retriever.get_data("metadata-year2.pkl")
metadata.rename(columns={"Unnamed: 0": "Attribute"}, inplace=True)

#vLoad the NZERTF data
year_2 = retriever.get_data("All-Subsystems-minute-Year2.pkl")

columns_to_clip = year_2.select_dtypes(np.number).columns #Only integer columns not string.
year_2[columns_to_clip].clip(lower=0) #Minimum value of column must be 0, else set to 0.

In [ ]:
watt_attributes = metadata[metadata["Units"] == "W"] # We only want attributes measured in Watts
watt_attributes = watt_attributes[watt_attributes['max_value'] > 0] #If no measurements are recorded in the attribute, we dont need to include it
watt_attributes = watt_attributes[watt_attributes["Parameter"] != "Power_Thermal"]
watt_attributes = watt_attributes[watt_attributes["Subsystem"] != "PV"]

consuming_attributes = watt_attributes["Attribute"].to_list()
print(len(consuming_attributes))

unwanted_attributes = ["Elec_PowerClothesWasher", "Elec_PowerRefrigerator",  "Elec_PowerMicrowave", "Elec_PowerPlugsLR", "Elec_PowerLights2ndFloor", "Elec_PowerPlugsBR3", "Elec_PowerPlugsBR2", "Elec_PowerPlugsInstKitD", "Elec_PowerHeatPumpWH2of2"]

#Se metadata for forklaring af unwanted_attributes
consuming_attributes = [attribute for attribute in consuming_attributes if attribute not in unwanted_attributes]

consuming_attributes

In [ ]:
energy_data = year_2[["Timestamp"] + consuming_attributes]

energy_data = energy_data[40265:40265*2]

In [ ]:
total_consumed = energy_data[consuming_attributes].sum().sum()
total_consumed * (1/60) / 1000

In [ ]:
total_consumed_HVAC = energy_data[['HVAC_DehumidifierPower', 'HVAC_HeatPumpIndoorUnitPower', 'HVAC_HeatPumpOutdoorUnitPower', 'Elec_PowerPlugsHVACInstrumentation']].sum().sum()
#HVACInstrumentation er et forsøg
total_consumed_HVAC * (1/60) / 1000

In [ ]:
total_consumed_DHW = energy_data[['DHW_HeatPumpWaterHeaterPowerTotal', 'DHW_SHWPumpsPowerWithStandby']].sum().sum()
total_consumed_DHW * (1/60) / 1000

In [ ]:
lighting_attributes = watt_attributes[watt_attributes["Subsystem"] == "Lighting"]
lighting_attributes = lighting_attributes["Attribute"].to_list()
lighting_attributes.remove('Elec_PowerLights2ndFloor')
print(lighting_attributes)

total_consumed_lighting = energy_data[lighting_attributes].sum().sum()
total_consumed_lighting * (1/60) / 1000

In [ ]:
total_consumed_appliances = energy_data[["Load_ClothesWasherPowerWithStandby", "Load_OvenPowerTotal", "Elec_PowerDishwasher", "Load_MicrowavePowerWithStandby", "Load_RefrigeratorPowerWithStandby", "Load_DryerPowerTotal"]].sum().sum()
#Mangler Cooktop?
total_consumed_appliances * (1/60) / 1000

In [ ]:
plug_attributes = watt_attributes[watt_attributes["Attribute"].str.contains("Plug")]
plug_attributes = plug_attributes[plug_attributes["Description"].str.contains("consumption")]
plug_attributes = plug_attributes["Attribute"].to_list()
#plug_attributes.remove('Elec_PowerPlugsLR')
plug_attributes.remove('Elec_PowerPlugsBR3')
plug_attributes.remove('Elec_PowerPlugsBR2')
plug_attributes.remove('Elec_PowerPlugsInstKitD')
plug_attributes.remove("Elec_PowerPlugsInstKitA")
plug_attributes.remove("Elec_PowerPlugsKitSink")
plug_attributes.remove("Elec_PowerPlugsKitRange")
plug_attributes.remove("Elec_PowerPlugsKitPeninsula")
plug_attributes.remove("Elec_PowerPlugsHVACInstrumentation")
plug_attributes.remove("Elec_PowerPlugsMBR") #Virker til at være subset af Load_MBRPlugLoadsPowerUsage
plug_attributes.remove("Elec_PowerPlugsInstBR3")
print(plug_attributes)

total_consumed_plugloads = energy_data[plug_attributes].sum().sum()
# total_consumed_plugloads = energy_data[["Elec_PowerPlugsDR", "Elec_PowerPlugsBR4", "Elec_PowerPlugsEntryHall","Load_BR2PlugLoadsPowerUsage", "Elec_PowerPlugsMBAEast", "Elec_PowerPlugsMBAWest", ""
#                                         "Load_LRPlugLoadsPowerUsage", "Load_BR3PlugLoadsPowerUsage", "Elec_PowerPlugsMBR", "Elec_PowerPlugsAttic"]].sum().sum()
total_consumed_plugloads * (1/60) / 1000

In [ ]:
# fig = px.line(data_frame=energy_data,
#               x = energy_data.index,
#               y = plug_attributes)
# fig.show()

In [ ]:
CONSUMED = total_consumed_plugloads + total_consumed_appliances + total_consumed_lighting + total_consumed_HVAC + total_consumed_DHW
CONSUMED * (1/60) / 1000